<a href="https://colab.research.google
.com/github/CrisLeaf/chatbot/blob/master/text_generation_with_gpt2.ipynb"
target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.15.0

     |████████████████████████████████| 3.4 MB 26.2 MB/s 
     |████████████████████████████████| 596 kB 64.7 MB/s 
     |████████████████████████████████| 3.3 MB 61.0 MB/s 
     |████████████████████████████████| 895 kB 60.8 MB/s 
     |████████████████████████████████| 67 kB 5.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!nvidia-smi

Sat Jan 15 21:47:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Prepare Dataset

In [3]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
import pandas as pd

df = pd.read_csv("/content/gdrive/My Drive/data-science/chatbot/text1.txt", sep=".", header=None).transpose()
df.columns = ["text"]
df["text"] = df["text"].apply(lambda x: str(x) + ". ")

In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=159)

In [6]:
data = ""
for line in train["text"]:
    data += line

In [7]:
def build_text_files(data_column, destination_path):
    with open(destination_path, "w") as f:
        data = ""
        for line in data_column:
            data += line
        f.write(data)

build_text_files(train["text"], "train_dataset.txt")
build_text_files(test["text"], "test_dataset.txt")

print("Train dataset length: " + str(len(train)))
print("Test dataset length: " + str(len(test)))

Train dataset length: 125
Test dataset length: 32


# Load the Model and the Trainer

## Load

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")
model = AutoModelForCausalLM.from_pretrained("DeepESP/gpt2-spanish")

Downloading:   0%|          | 0.00/115 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/914 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/821k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/262 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

In [9]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=128
    )

    test_dataset = TextDataset(
        tokenizer=tokenizer, 
        file_path=test_path,
        block_size=128
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset("train_dataset.txt", "test_dataset.txt", 
                                                          tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


## Train

In [15]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/gdrive/My Drive/data-science/chatbot/gpt2-climate",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_steps=40,
    save_steps=80,
    warmup_steps=50,
    prediction_loss_only=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


# Train and Save the Model

In [16]:
trainer.train()

***** Running training *****
  Num examples = 29
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 10


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=10, training_loss=4.9016273498535154, metrics={'train_runtime': 9.5319, 'train_samples_per_second': 30.424, 'train_steps_per_second': 1.049, 'total_flos': 18943672320000.0, 'train_loss': 4.9016273498535154, 'epoch': 10.0})

In [21]:
trainer.save_model()
tokenizer.save_pretrained("/content/gdrive/My Drive/data-science/chatbot/gpt2-climate")

Saving model checkpoint to /content/gdrive/My Drive/data-science/chatbot/gpt2-climate
Configuration saved in /content/gdrive/My Drive/data-science/chatbot/gpt2-climate/config.json
Model weights saved in /content/gdrive/My Drive/data-science/chatbot/gpt2-climate/pytorch_model.bin
tokenizer config file saved in /content/gdrive/My Drive/data-science/chatbot/gpt2-climate/tokenizer_config.json
Special tokens file saved in /content/gdrive/My Drive/data-science/chatbot/gpt2-climate/special_tokens_map.json


('/content/gdrive/My Drive/data-science/chatbot/gpt2-climate/tokenizer_config.json',
 '/content/gdrive/My Drive/data-science/chatbot/gpt2-climate/special_tokens_map.json',
 '/content/gdrive/My Drive/data-science/chatbot/gpt2-climate/vocab.json',
 '/content/gdrive/My Drive/data-science/chatbot/gpt2-climate/merges.txt',
 '/content/gdrive/My Drive/data-science/chatbot/gpt2-climate/added_tokens.json',
 '/content/gdrive/My Drive/data-science/chatbot/gpt2-climate/tokenizer.json')

# Test the Model

In [22]:
from transformers import pipeline

bot = pipeline("text-generation", 
               model="/content/gdrive/My Drive/data-science/chatbot/gpt2-climate", 
               tokenzier="DeepESP/gpt2-spanish")

loading configuration file /content/gdrive/My Drive/data-science/chatbot/gpt2-climate/config.json
Model config GPT2Config {
  "_name_or_path": "/content/gdrive/My Drive/data-science/chatbot/gpt2-climate",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
  

In [25]:
bot("Las consecuencias del cambio climático son")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Las consecuencias del cambio climático son que los elementos hidrocarburos que afectan a las personas enferman en lugar de afectar de manera específica los beneficios de los organismos vivos. La necesidad de grandes burbujas en el clima mundial requiere la reducción de los gases y la necesidad'}]